# Experimento

## Importação de dados de artistas

### Dataset aset400
Eles utilizaram a versão ** filtrada ** do dataset.

In [110]:
import networkx as nx
from random import shuffle

In [78]:
# loading the artist ids
with open('aset400.3-canon-musicseer.ids', 'r') as f:
    artist_names = {}
    for i in f.read().split("\n"):
        name, id = i.partition(" ")[::2]
        if(id != ''):
            artist_names[int(id)] = name
    print("Total imported: %i artists" % len(artist_names.keys()))

Total imported: 400 artists


In [86]:
# loading filtered triplets
keys = artist_names.keys()
with open('filtered-survey.txt', 'r') as f:
    artist_triplets = []
    for i in f.read().split("\n"):
        trip = i.split(" ")[-3:]
        if(trip[0] != ''):
            trip = [int(x) for x in trip]
            if (trip[0] in keys) & (trip[1] in keys) & (trip[2] in keys):
                 artist_triplets.append(trip)
                
    print("Imported: %i triplets" % len(artist_triplets))
    


Imported: 16243 triplets


## Filtering
We filter out all measurements (i, j, k) if (i, k, j) is also present.

In [87]:
# Grafo sem a filtragem (i, j, k) (i, k, j)
G = nx.DiGraph()

for trip in artist_triplets:
    node1 = (trip[0], trip[1])
    node2 = (trip[0], trip[2])
    G.add_node(node1)
    G.add_node(node2)
    G.add_edge(node1, node2)
    
print("Nodes:", G.__len__())
print("Edges:",G.size())


Nodes: 5252
Edges: 10609


In [80]:
for trip in artist_triplets:
    tmp = [trip[0], trip[2], trip[1]]
    if tmp in artist_triplets:
        artist_triplets.remove(tmp)
        artist_triplets.remove(trip)
        
print(len(artist_triplets))

14327


In [88]:
# Grafo com a filtragem (i, j, k) (i, k , j)
G = nx.DiGraph()

for trip in artist_triplets:
    node1 = (trip[0], trip[1])
    node2 = (trip[0], trip[2])
    G.add_node(node1)
    G.add_node(node2)
    G.add_edge(node1, node2)
    
print("Nodes:", G.__len__())
print("Edges:",G.size())

Nodes: 5252
Edges: 10609


## Máximo subgrafo acíclico

In [126]:
def max_subDAG(G):
    G_ = nx.DiGraph()

    for node in nx.nodes(G):
        G_.add_node(node)

    edges_shuf = nx.edges(G)
    shuffle(edges_shuf)

    for edge in edges_shuf:
        G_.add_edge(edge[0], edge[1])
        if not nx.is_directed_acyclic_graph(G_):
            G_.remove_edge(edge[0], edge[1])
    return G_

def n_trials(n, G):
    graphs = []
    G_n = nx.DiGraph()

    for i in range(n):
        graphs.append(nx.DiGraph())
    for i in range(n):
        graphs[i] = max_subDAG(G)
    
    edges = nx.edges(graphs[0])

    for i in range(1, n):
        edges_ = nx.edges(graphs[i])
        edges = list(set(edges).intersection(edges_))
    
    for node in nx.nodes(G):
        G_n.add_node(node)
        
    for edge in edges:
        G_n.add_edge(edge[0], edge[1])
    
    return G_n

G_1 = n_trials(10, G)

print("Nodes:", G_1.__len__())
print("Edges:",G_1.size())

Nodes: 5252
Edges: 7632


In [138]:
edges = open('subgraph.txt', 'w')

for edge in G_1.edges():
    edges.write("%s\n" % str(edge))